In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
import math
import time as time
import itertools


KeyboardInterrupt: 

In [ ]:
df = pd.read_csv('/Users/guillermotafur/Desktop/Data Project/fifa.csv')
df

: 

In [ ]:
df['position'] = np.where(df['position'] == 'CB', 'Defender', df['position'])
df['position'] = np.where(df['position'] == 'LB', 'Defender', df['position'])
df['position'] = np.where(df['position'] == 'RB', 'Defender', df['position'])
df['position'] = np.where(df['position'] == 'RWB', 'Defender', df['position'])
df['position'] = np.where(df['position'] == 'LWB', 'Defender', df['position'])
df['position'] = np.where(df['position'] == 'GK', 'Defender', df['position'])

: 

In [ ]:
df['league_name'] = np.where(df['league_name'] == 'French Ligue 1', 'yes', df['league_name'])
df['league_name'] = np.where(df['league_name'] == 'German 1. Bundesliga', 'yes', df['league_name'])
df['league_name'] = np.where(df['league_name'] == 'English Premier League', 'yes', df['league_name'])
df['league_name'] = np.where(df['league_name'] == 'Spain Primera Division', 'yes', df['league_name'])
df['league_name'] = np.where(df['league_name'] == 'Italian Serie A', 'yes', df['league_name'])
df['league_name'] = np.where(df['league_name'] == 'Campeonato Brasileiro S√©rie A', 'yes', df['league_name'])
df['league_name'] = np.where(df['league_name'] == 'Portuguese Liga ZON SAGRES', 'yes', df['league_name'])
df['league_name'] = np.where(df['league_name'] == 'Holland Eredivisie', 'yes', df['league_name'])
df['league_name'] = np.where(df['league_name'] == 'Mexican Liga MX', 'yes', df['league_name'])
df['league_name'] = np.where(df['league_name'] == 'Turkish S√ºper Lig', 'yes', df['league_name'])

: 

In [ ]:
fifa = df.loc[df['position'] != 'Defender']
fifa = fifa.loc[fifa['league_name'] == 'yes']
fifa

: 

In [ ]:
train = fifa.sample(frac=0.8)
test = fifa.drop(train.index)

: 

In [ ]:
df.corr()

: 

In [ ]:
sns.regplot(x = 'weight', y = 'pace', data = df,scatter_kws={"color": "orange"}, line_kws={"color": "blue"}) #Visualizing data


: 

In [ ]:
fifa.describe()

: 

In [ ]:
model = sm.ols(formula = 'np.log(value_eur)~pace+shooting+passing+dribbling+physic+age+height+weight', data = train).fit()
model.summary()

: 

In [ ]:
model2 = sm.ols(formula = 'np.log(value_eur)~shooting+passing+dribbling+skill_moves+height+physic+age', data = train).fit()
model2.summary()

: 

In [ ]:
out = model2.outlier_test()
out

: 

In [ ]:
sns.scatterplot(x = (model2.fittedvalues), y=(out.student_resid),color = 'orange')
sns.lineplot(x = [model2.fittedvalues.min(),model2.fittedvalues.max()],y = [0,0],color = 'blue')
plt.xlabel('Fitted values')
plt.ylabel('Studentized Residuals')

: 

In [ ]:
np.sum((np.abs(out.student_resid)>3))

: 

In [ ]:
influence = model2.get_influence()
leverage = influence.hat_matrix_diag
average_leverage = (model2.df_model+1)/model2.nobs
average_leverage
high_leverage_threshold = 4*average_leverage
np.sum(leverage>high_leverage_threshold)


: 

In [ ]:
fifafiltered = train.drop(np.intersect1d(np.where(np.abs(out.student_resid)>3)[0],
                                           (np.where(leverage>high_leverage_threshold)[0])))

: 

In [ ]:
modelfiltered = sm.ols(formula = 'np.log(value_eur)~shooting+passing+dribbling+skill_moves+height+physic+age+(age*physic)+(dribbling*passing)+I(skill_moves**2)', data = fifafiltered).fit()
modelfiltered.summary()

: 

In [ ]:
prediction = modelfiltered.predict(test)
print(np.sqrt(((test.value_eur - np.exp(prediction))**2).mean()))

: 

In [ ]:
def processSubset(predictor_subset):
    # Fit model on feature_set and calculate R-squared
    model = sm.ols('value_eur~' + '+'.join(predictor_subset),data = train).fit()
    Rsquared = model.rsquared
    return {"model":model, "Rsquared":Rsquared}

def getBest_model(k):
    tic = time.time()
    results = []
    for combo in itertools.combinations(X.columns, k):
        results.append(processSubset((list(combo))))

    # Wrap everything up in a dataframe
    models = pd.DataFrame(results)

    # Choose the model with the highest RSS
    best_model = models.loc[models['Rsquared'].argmax()]
    
    toc = time.time()
    print("Processed", models.shape[0], "models on", k, "predictors in", (toc-tic), "seconds.")
    return best_model

: 

In [ ]:
X = train[['pace','shooting','passing','dribbling','physic','age']]
X_backup = train[['pace','shooting','passing','dribbling','physic','age']]

: 

In [ ]:
for combo in itertools.combinations(X_backup.columns, 2):    
    train['_'.join(combo)] = train[combo[0]]*train[combo[1]]
    test['_'.join(combo)] = test[combo[0]]*test[combo[1]]
    X.loc[:,'_'.join(combo)] = train.loc[:,'_'.join(combo)]

: 

In [ ]:

models_best = pd.DataFrame(columns=["Rsquared", "model"])

tic = time.time()
for i in range(1,1+X.shape[1]):
    models_best.loc[i] = getBest_model(i)

toc = time.time()
print("Total elapsed time:", (toc-tic), "seconds.")

: 

In [ ]:
def best_sub_plots():
    plt.figure(figsize=(20,10))
    plt.rcParams.update({'font.size': 18, 'lines.markersize': 10})

    # Set up a 2x2 grid so we can look at 4 plots at once
    plt.subplot(2, 2, 1)

    # We will now plot a red dot to indicate the model with the largest adjusted R^2 statistic.
    # The argmax() function can be used to identify the location of the maximum point of a vector
    plt.plot(models_best["Rsquared"])
    plt.xlabel('# Predictors')
    plt.ylabel('Rsquared')

    # We will now plot a red dot to indicate the model with the largest adjusted R^2 statistic.
    # The argmax() function can be used to identify the location of the maximum point of a vector

    rsquared_adj = models_best.apply(lambda row: row[1].rsquared_adj, axis=1)

    plt.subplot(2, 2, 2)
    plt.plot(rsquared_adj)
    plt.plot(1+rsquared_adj.argmax(), rsquared_adj.max(), "or")
    plt.xlabel('# Predictors')
    plt.ylabel('adjusted rsquared')

    # We'll do the same for AIC and BIC, this time looking for the models with the SMALLEST statistic
    aic = models_best.apply(lambda row: row[1].aic, axis=1)

    plt.subplot(2, 2, 3)
    plt.plot(aic)
    plt.plot(1+aic.argmin(), aic.min(), "or")
    plt.xlabel('# Predictors')
    plt.ylabel('AIC')

    bic = models_best.apply(lambda row: row[1].bic, axis=1)

    plt.subplot(2, 2, 4)
    plt.plot(bic)
    plt.plot(1+bic.argmin(), bic.min(), "or")
    plt.xlabel('# Predictors')
    plt.ylabel('BIC')
best_sub_plots()

: 

In [ ]:
best_subset_model = models_best.loc[7,'model']
models_best.loc[7,'model'].summary()

: 